### Create Or Update : 
```
INSERT INTO users (id, name, email)
VALUES (123, 'John Doe', 'john.doe@example.com')
ON DUPLICATE KEY UPDATE name = 'John Doe', email = 'john.doe@example.com';

* but must raise an error in insert like unique index;


laravel : 

\Illuminate\Support\Facades\DB::enableQueryLog();

    \App\Models\User::query()->UpdateOrCreate(
        [
            'name'=>'masoud',
            'email' => 'masoud@gmail.com',
        ],
        [
            'password' => '123_2'
        ]
    )->tosql();

dd(\Illuminate\Support\Facades\DB::getQueryLog());


```


### Drop duplicated : 
```
SELECT DISTINCT city FROM Employee;

SELECT COUNT(DISTINCT country) FROM country;

```

### Operator : AND, OR, In, Not In, BETWEEN, Null
```
SELECT COUNT(*) FROM `Invoice`
WHERE NOT (InvoiceDate > '2013-01-01' and InvoiceDate < '2013-12-30');

SELECT count(*) FROM `payment`
WHERE payment_date BETWEEN '2005-05-26' and '2005-05-27';

SELECT * FROM `address`
WHERE district in ("Texas","QLD");

```

### Like, REGEXP Operator:

 - % : any    character
 - _ : single character
  - 
```
SELECT * FROM `actor` WHERE last_name LIKE 'H____S';
SELECT * FROM `actor` WHERE last_name REGEXP 'pattern'
```

### OrderBy
```
SELECT * FROM `rental` ORDER BY inventory_id + customer_id ASC;
```

### Limit
```
SELECT * FROM `actor` LIMIT 4(from),5(count);
```



# Combine:
    - Combine Columns : Join
    - Combine Rows    : Union 

### Composite Primary key Join
```
SELECT * FROM rental r
JOIN payment p 
	ON p.payment_id = r.rental_id
    AND p.customer_id = r.inventory_id
```

### Implicit join, and the USING
```
SELECT c.customer_id, sum(p.amount) FROM payment p, customer c
WHERE p.customer_id = c.customer_id
GROUP BY c.customer_id;


SELECT p.payment_id, c.customer_id, p.amount 
FROM customer c
JOIN payment p 
USING (customer_id);


```

### OuterJoin: Left, Right ------- InnerJoin: join


### Unioin:
```
SELECT 
	r.rental_id,
    r.rental_date,
    'Active' as status
FROM rental r
WHERE r.rental_date >= '2005-06-01' and r.rental_date  < '2005-06-30'
UNION
SELECT 
	r.rental_id,
    r.rental_date,
    'Deactive' as status
FROM rental r
WHERE r.rental_date >= '2005-05-01' and r.rental_date  < '2005-05-30';
```


### Insert Single:
```
INSERT INTO actor 
VALUES (DEFAULT, 'masoud', 'nazarpoor', NOW())

INSERT INTO actor (first_name, last_name)
VALUES ('mostafa', 'nazarpoor')

```

### Insert multiple
```
INSERT INTO actor (first_name, last_name)
VALUES 
	('younes', 'nazarpoor'),
    ('mamad', 'nazarpoor')
    
```

### Insert Hierarchical
```
INSERT INTO country (country)
VALUES ('NazarAbadxxxx');

INSERT INTO city (city, country_id)
VALUES ('HasirAbad', LAST_INSERT_ID() )
```

### Creating a copy of the table
```
CREATE TABLE country_archive AS
SELECT * FROM country
```

### SubQueries with insert
```
INSERT INTO country_archive
SELECT * FROM country WHERE country_id > 10
```

### Remove All Data with Truncate
```
TRUNCATE country_archive;
```

### Update
```
UPDATE country_archive 
SET 
	last_update = DEFAULT,
    country = NULL
WHERE country_id > 20
```

### Delete
```
DELETE FROM country
where id > 120
```

### Aggregate Functions
```
MAX(), MIN(), AVG(), SUM(), COUNT()

SELECT 
    COUNT(DISTINCT customer_id)
FROM `payment`;


```

### GROUP BY
```
SELECT 
    t1.name,
    sum(t2.length)/60
from category t1
left join film_category tr on tr.category_id = t1.category_id
inner join film t2 on t2.film_id = tr.film_id
group by t1.name
having sum(t2.length)/60 > 110
            
            
SELECT 
    customer_id,
    staff_id,
    sum(amount) as sum_of_paid,
    count(*)
from payment
group by customer_id, staff_id
                
                

```

# filter data:
    ## before group data : where (select any column)
    ## after group data : having (select only selected column)


### ROLLUP : summery value for each group
```
SELECT 
	customer_id,
    staff_id,
    sum(amount) as sum_of_paid, 
    count(*)
from payment
group by customer_id, staff_id WITH ROLLUP;

```

### subqueries

```
select * from customer
where customer_id in (
    SELECT customer_id from payment t1
    where t1.amount = (
        select max(amount) from payment
    )
)

```


### Keyword: ALL
```
SELECT * FROM payment
WHERE amount > (
    SELECT max(amount) FROM `payment`
    WHERE customer_id = 10
);

SELECT * FROM payment
WHERE amount > ALL (
    SELECT amount FROM `payment`
    WHERE customer_id = 10
);

```

### Keyword: ANY
```
SELECT * FROM `customer`
WHERE customer_id IN (
SELECT 
    customer_id
    FROM payment
    GROUP BY customer_id
    HAVING COUNT(*) > 40
);

SELECT * FROM `customer`
WHERE customer_id = ANY (
SELECT 
    customer_id
    FROM payment
    GROUP BY customer_id
    HAVING COUNT(*) > 40
);

```


### xxx
#### customer with paid more than own city paid
```
SELECT 
	c.customer_id,
    avg(p.amount) customer_avg_amount,
    cavg.avg city_avg_amount
FROM customer c
INNER JOIN payment p USING (customer_id)
INNER JOIN address a USING (address_id)
INNER JOIN city ct USING (city_id)
INNER JOIN (
    SELECT 
        ct.city_id,
        avg(p.amount) avg
    FROM customer c
    INNER JOIN address a USING (address_id)
    INNER JOIN city ct USING (city_id)
    INNER JOIN payment p USING (customer_id)
    GROUP BY ct.city_id
) cavg USING (city_id)
GROUP BY c.customer_id
HAVING customer_avg_amount > cavg.avg
ORDER BY c.customer_id;

```

### Corelated SubQuery
#### refrence subquery to parent query
```
Noraml :

    SELECT * 
    FROM payment p
    JOIN (
        SELECT 
            customer_id,
            AVG(amount) avg
        FROM payment p
        GROUP BY customer_id
    ) ca USING (customer_id)
    WHERE p.amount > ca.avg;


Corelated:

    SELECT *
    FROM payment p
    WHERE amount > (
        SELECT AVG(amount)
        FROM payment
        WHERE customer_id = p.customer_id
    )
    ORDER BY customer_id,amount;
    
```

### keword: EXIST
```
SELECT * 
FROM customer
WHERE customer_id IN (
	SELECT DISTINCT customer_id 
	FROM payment
)

SELECT * 
FROM customer c
WHERE EXISTS (
	SELECT DISTINCT customer_id 
	FROM payment
    WHERE customer_id = c.customer_id
); # performance is better

```

### SubQuery in SELECT
```
SELECT 
	customer_id,
    amount,
    (SELECT AVG(amount) FROM payment) total_avg,
    amount - (SELECT total_avg ) amount_avg_diff
FROM payment;

```

### Numeric Built-in Functions
```
SELECT ROUND(5.56,1); ===> 5.6
SELECT CEILING(5.56); ===> 6
SELECT FLOOR(5.56);   ===> 5
SELECT ABS(-5.56);    ===> 5.6
SELECT RAND();        ===> return random number 0~1
```


### String Built-in Functions
```
SELECT LENGTH('sky');
SELECT UPPER('sky');
SELECT LOWER('sky');
SELECT LTRIM('sky');
SELECT RTRIM('sky');
SELECT TRIM('sky');
SELECT LEFT('masoud',2);     ===> return ma
SELECT RIGHT('masoud',2);    ===> return ud
SELECT SUBSTRING('masoud nazarpoor',1,6); ===> return masoud
SELECT REPLACE('masoud nazarpoor', 'masoud', 'younes'); ===> return younes nazarpoor
SELECT CONCAT(first_name, ' ', last_name) as full_name FROM `customer`;
```


### DateTime Built-in Functions
```
SELECT NOW(), CURDATE(), CURTIME();
SELECT YEAR('2020-12-05'), MONTH('2020-12-05'), DAY('2020-12-05');
SELECT HOUR(NOW()), MINUTE(NOW()), SECOND(NOW());
SELECT MONTHNAME('2020-12-05'), DAYNAME('2020-12-05');
SELECT DATE_FORMAT( NOW(), '%Y-%m-%d %h:%i:%S %p');

SELECT DATE_ADD( NOW(), INTERVAL 1 YEAR);
SELECT DATEDIFF('2018-01-10', '2019-12-15');
SELECT TIME_TO_SEC('00:02:00'); 
NOW() - INTERVAL 1 YEAR
```

### IFNULL - COALESCE - IF
```
SELECT 
	payment_id,
    amount,
    IFNULL( rental_id, 'NOTSET' )
FROM `payment`;

SELECT 
	payment_id,
    amount,
    COALESCE( rental_id, amount, 'NOTSET' )
FROM `payment`;

SELECT 
	payment_id,
    amount,
    IF( amount>5, 'EXPENSIVE', 'CHEAP') as status,
    IF( YEAR(payment_date) = '2006', 'ACTIVE', 'ARCHIVED')
FROM `payment`;
```


### CASE
```
SELECT 
	payment_id,
    amount,
    CASE
		WHEN amount < 3 THEN 'CHEAP'
        WHEN amount > 3 AND amount < 5 THEN 'NORMAL'
        ELSE 'EXPENSIVE'
    END as status
FROM `payment`;

```



### VIEW
```
CREATE OR REPLACE VIEW cities_average_payments AS
SELECT 
   ROW_NUMBER() OVER ( ORDER BY country ) AS id,
   ct.city_id,
   ct.city city,
   cnt.country,
   avg(p.amount) payment_average
FROM customer c
LEFT JOIN address a USING (address_id)
LEFT JOIN city ct USING (city_id)
LEFT JOIN country cnt USING (country_id)
LEFT JOIN payment p USING (customer_id)
GROUP BY ct.city_id;


DROP VIEW cities_average_payments
```

### PROCEDURE
```
DELIMITER $$
CREATE PROCEDURE get_city_customers
(
	city_id INT	
)
BEGIN
	IF city_id IS NULL THEN
    	SET city_id = 463;
    END IF;
    
    IF city_id <=0 THEN
        SIGNAL SQLSTATE '22003' SET MESSAGE_TEXT = 'invalid city_id';
    END IF;

	SELECT c.* 
    FROM customer c
    LEFT JOIN address a USING (address_id)
    LEFT JOIN city ct USING (city_id)
    WHERE ct.city_id = city_id;
END $$
DELIMITER ; 



DELIMITER $$
CREATE PROCEDURE get_city_customers
(
	city_id INT	
)
BEGIN
	SELECT c.* 
    FROM customer c
    LEFT JOIN address a USING (address_id)
    LEFT JOIN city ct USING (city_id)
    WHERE ct.city_id = IFNULL( city_id, ct.city_id );
END $$
DELIMITER ;



## Run  : CALL get_city_customers();
## Drop : DROP PROCEDURE IF EXISTS get_city_customers



### ----------------WITH OUT PARAMETERS

DELIMITER $$
CREATE PROCEDURE get_city_customers
(
	city_id INT,
    OUT customers_count INT
)
BEGIN
	IF city_id IS NULL THEN
    	SET city_id = 463;
    END IF;
    
    IF city_id <=0 THEN
        SIGNAL SQLSTATE '22003' SET MESSAGE_TEXT = 'invalid city_id';
    END IF;

	SELECT 
    COUNT(*) INTO customers_count
    FROM customer c
    LEFT JOIN address a USING (address_id)
    LEFT JOIN city ct USING (city_id)
    WHERE ct.city_id = city_id;
END $$
DELIMITER ; 

## Run :    SET @customers_count = 0;
            CALL get_city_customers(20, @customers_count);
            SELECT @customers_count


```


### Local Variable
```
DELIMITER $$
CREATE PROCEDURE get_city_customers_counts()
BEGIN
	DECLARE customer_count INT DEFAULT 0;
    
	SELECT COUNT(*) 
    INTO customer_count
    FROM customer c
    LEFT JOIN address a USING (address_id)
    LEFT JOIN city ct USING (city_id)
    WHERE ct.city_id = 463;
	
    SET customer_count = customer_count + 1;
    SELECT customer_count;
END $$
DELIMITER ; 


```





### Functions
```
### just return single values

DELIMITER $$
CREATE FUNCTION sum_of_customer_paid
(
    customer_id INT
)
RETURNS DECIMAL(9,2)
READS SQL DATA
BEGIN
    DECLARE sum_of_amount DECIMAL(9,2) DEFAULT 0.0;
    
	SELECT 
    	SUM(p.amount) INTO sum_of_amount
    FROM payment p
    WHERE p.customer_id = customer_id;
    
	RETURN IFNULL(sum_of_amount, 0);
END $$
DELIMITER ; 

DROP FUNCTION IF EXISTS sum_of_customer_paid

SELECT sum_of_customer_paid(22);

SELECT 
 customer_id,
 sum_of_customer_paid(customer_id)
FROM payment
GROUP BY customer_id;
```




### Triggers
```
DELIMITER $$

CREATE TRIGGER after_insert_payment
	AFTER/BEFORE INSERT/UPDATE/DELETE on payment
    FOR EACH ROW
BEGIN

	UPDATE payment
    SET amount = NEW.amount + (NEW.amount * .5) 
	WHERE payment_id = NEW.payment_id;

END $$
DELIMITER ; 

### OLD for deleted row
### SHOW TRIGGERS
### DROP TRIGGERS IF EXISTS after_insert_payment

```




### Events (schedule )
```
SHOW VARIABLES LIKE 'event%';
SHOW EVENTS
SET GLOBAL event_scheduler = OFF


------------- CREATE EVENT

DELIMITER $$
CREATE EVENT yearly_insert_category
ON SCHEDULE
	-- AT '2020-12-12'
    EVERY 10 SECOND START NOW() ENDS NOW() + INTERVAL 1 DAY
DO BEGIN

	INSERT INTO category
    VALUES (
        DEFAULT,
        CONCAT('CAT_',NOW()),
        DEFAULT
        );

END $$
DELIMITER ;

------------- DISABLE EVENT

ALTER EVENT yearly_insert_category DISABLE


------------- UPDATE EVENT

DELIMITER $$
ALTER EVENT yearly_insert_category
ON SCHEDULE
	-- AT '2020-12-12'
    EVERY 10 SECOND START NOW() ENDS NOW() + INTERVAL 1 DAY
DO BEGIN

	INSERT INTO category
    VALUES (
        DEFAULT,
        CONCAT('CAT_',NOW()),
        DEFAULT
        );

END $$
DELIMITER ;


------------- DROP EVENT

DROP EVENTS IF EXISTS yearly_insert_category


```





### Transaction

<img src='../img/Screenshot from 2023-04-02 13-10-35.png'/>
    ACID :
        - Atomicity : all done or rollback
        - Consistency
        - Isolation
        - Durability : when transaction commited if systemdown or enything happen, garunty not loss changes
* Default Transaction Level In MYSQL : REPEATABLE READ
```

START TRANSACTION;

	INSERT INTO category
    VALUES (DEFAULT,'cat_0002',DEFAULT);
    
    INSERT INTO film_category
    VALUES (1,LAST_INSERT_ID(),DEFAULT);
COMMIT/ROLLBACK;


Isolation Levels:
    - READ UNCOMMITED
    - READ COMMITED
    - REPEATABLE READ
    - SERIALIZABLE



#### * SHOW VARIABLES LIKE 'autocommit'
#### * SET transaction isolation level serializable
#### * SET session transaction isolation level serializable
#### * SET global transaction isolation level serializable
```


### DeadLock : 
    * happened when transactions hold to each other
    - check order of your code



### Data Types

```
   String:
    - CHAR(x) fixed string
    - VARCHAR(x) max: 65,535 character ~65KB
    - MEDIUMTEXT 16MB
    - LONGTEXT 4G
    - TINYTEXT 255Bytes
    - TEXT 64KB
    
   Number:
    - TINYINT 1B [-128,127]
    - UNSIGNED TINYINT  [0,255]
    - SMALLINT 2b [-32K,32K]
    - MEDIUMINT 3b [-8M,8M]
    - INT 4b [-2B,2B]
    - BIGINT 8b [-9Z,9Z]
    - DECIMAL(9,2) 1234567,89
    - FLOAT
    - DOUBLE
    
   Enum-Set:
    - ENUM('one','two')
    - SET
    
   Date/Time:
    - DATE
    - TIME
    - DATETIME
    - TIMESTAMP
    - YEAR
    
   Blob:
      - store image/video... any binary data
      - TINYBLOB 255b
      - BLOB 65K
      - MEDIUMBLOB  16m
      - LONGBLOB 4G
    
    
   Json:
   
    UPDATE actor
    SET
        attrs = JSON_OBJECT(
        	'age', 27,
            'scores', JSON_ARRAY(10,22.3,50),
            'actors', JSON_ARRAY(
				JSON_OBJECT('name','jan','age',27),  
                JSON_OBJECT('name','scarlet','age',20)
            ),
            'car', JSON_OBJECT('name','pride','color','red')
        )
    WHERE actor_id = 1;
    
    
    UPDATE actor
    SET
        attrs = JSON_SET(
        	attrs,
            '$.age',30
        )
    WHERE actor_id = 1;
    
    
    UPDATE actor
    SET
        attrs = JSON_REMOVE(
        	attrs,
            '$.age'
        )
    WHERE actor_id = 1;
    
    
    
    SELECT 
        actor_id,
        JSON_EXTRACT(attrs, '$.age') 
    FROM actor
    WHERE 1
    
    
    SELECT 
        actor_id,
        JSON_EXTRACT(attrs, '$.car.name') 
    FROM actor
    WHERE 1;
    
    
    SELECT 
        actor_id,
        JSON_EXTRACT(attrs, '$.actors[1].name') 
    FROM `actor`
    WHERE 1;
    

    SELECT 
        actor_id,
        attrs ->> '$.actors[1].name' 
    FROM actor
    WHERE 1;

    SELECT 
        actor_id,
        first_name
    FROM actor
    WHERE attrs ->> '$.actors[1].name' = 'scarlet';

```



### INDEXES 
    - indexes slow down insert/update in table but speed up read
    - for each index created one btree
    - composite index { column1, column2 }
    
    - Primary Key             : a unique identify column/or/multiple column for each rows and not changed
    - Foreign Keys            : is a column in one table that refrences to primary key in another table
    - Foreign Key Constraints : onDelete onUpdate
    
                                - restrict: If there are any rows in the child table that reference the row in the                                             parent table, the action (delete/update) will not be executed
                                
                                - cascade:  It will delete or update all the rows in the child table that                                                     reference the row in the parent table.




### Normalization

 - each column should be single value
 - every table should describe one entity and every column in that table should describe that entity
 - when we have firstname and lastname column we dont need to full_name column




### CREATE DATABASE IF NOT EXISTS TBLNAME;
### DROP   DATABASE IF EXISTS TBLNAME;
### ALTER TABLE customer
    ADD fullname  VARCHAR(50) NOT NULL AFTER firstname,
    MODIFY COLUMN city_id INT DEFAULT '***',
    DROP points ;


### SHOW CHARSET;
    utf8 3b ==> CHAR(10) ==> 10 * 3 = 30bytes






